In [ ]:
email_project/
│
├── send_email.py             # Standalone Python script to send email
├── .env                      # Environment variables file to store credentials
└── templates/                # Folder for HTML templates
    └── email_template.html   # HTML template for the email

In [ ]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from dotenv import load_dotenv
from jinja2 import Template

# Load environment variables from .env file
load_dotenv()

# Email configuration
smtp_server = 'smtp.office365.com'
smtp_port = 587
username = os.getenv('OUTLOOK_MAIL_USERNAME')
password = os.getenv('OUTLOOK_MAIL_PASSWORD')

# Read the HTML template
with open('templates/email_template.html', 'r') as file:
    template = Template(file.read())

# Render the template with dynamic content
html_content = template.render(title='Test Email', message='This is an email sent securely using a Python script.')

# Create the email message
msg = MIMEMultipart('alternative')
msg['From'] = username
msg['To'] = 'recipient@example.com'  # Replace with the recipient's email
msg['Subject'] = 'Secure Test Email with Python Script'

# Attach the HTML content to the email
msg.attach(MIMEText(html_content, 'html'))

# Send the email
try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()  # Start a secure TLS connection
    server.login(username, password)
    server.sendmail(msg['From'], msg['To'], msg.as_string())
    server.quit()
    print('Email successfully sent!')
except Exception as e:
    print(f'Error sending email: {e}')


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>{{ title }}</title>
</head>
<body>
    <h1>{{ title }}</h1>
    <p>{{ message }}</p>
</body>
</html>


In [ ]:
OUTLOOK_MAIL_USERNAME=your_email@outlook.com
OUTLOOK_MAIL_PASSWORD=your_password


In [ ]:
mkdir email_project
cd email_project
python -m venv venv
source venv/bin/activate  # On Linux/macOS
venv\Scripts\activate     # On Windows

In [ ]:
pip install python-dotenv Jinja2

In [ ]:
python send_email.py

## flask

structure for your Flask project

In [ ]:
flask_mail_project/
│
├── app.py                        # Main Flask application file
├── .env                          # Environment variables file to store credentials
├── logs/                         # Folder where log files will be stored
├── templates/                    # Folder for HTML templates
│   ├── welcome_email.html        # HTML template for welcome email
│   ├── password_reset.html       # HTML template for password reset email
│   └── newsletter.html           # HTML template for newsletter email
├── data/                         # Folder for data files
│   ├── json/                     # Subfolder for JSON files
│   │   └── email_data.json       # Example JSON file with email data
│   └── sql/                      # Subfolder for SQL scripts
│       └── fetch_email_data.sql  # SQL script for fetching email data
├── recipients/                   # Folder for recipient lists
│   └── recipients.txt            # Plain text file containing the recipients' emails
├── scripts/                      # Folder for Bash scripts
│   └── send_emails.sh            # Bash script to automate email sending
└── requirements.txt              # Project dependencies file

In [ ]:
app.py

In [ ]:
from flask import Flask, render_template
from flask_mail import Mail, Message
import os
import sys
import json
import sqlite3
import logging
from logging.handlers import RotatingFileHandler
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Create logs directory if it doesn't exist
if not os.path.exists('logs'):
    os.makedirs('logs')

# Set up logging
log_filename = datetime.now().strftime("logs/log_%Y-%m-%d_%H-%M-%S.log")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
handler = RotatingFileHandler(log_filename, maxBytes=10000000, backupCount=5)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

app = Flask(__name__)

# Configure Flask-Mail for Outlook
app.config['MAIL_SERVER'] = 'smtp.office365.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USE_SSL'] = False
app.config['MAIL_USERNAME'] = os.getenv('OUTLOOK_MAIL_USERNAME')
app.config['MAIL_PASSWORD'] = os.getenv('OUTLOOK_MAIL_PASSWORD')
app.config['MAIL_DEFAULT_SENDER'] = os.getenv('OUTLOOK_MAIL_USERNAME')

# Initialize Flask-Mail
mail = Mail(app)

def fetch_data_from_json(json_file):
    # Load data from the specified JSON file
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
        logger.info(f"Successfully loaded data from {json_file}")
        return data
    except Exception as e:
        logger.error(f"Error reading JSON file {json_file}: {e}")
        return None

def fetch_data_from_db(query_file):
    # Load SQL query from the file
    try:
        with open(query_file, 'r') as file:
            query = file.read()
    except Exception as e:
        logger.error(f"Error reading SQL file {query_file}: {e}")
        return None

    # Execute SQL query to fetch data
    try:
        conn = sqlite3.connect('example.db')  # Replace with your database file
        conn.row_factory = sqlite3.Row  # This line allows us to use column names as keys
        cursor = conn.cursor()
        cursor.execute(query)
        row = cursor.fetchone()
        conn.close()

        if row:
            data = dict(row)  # Convert the row to a dictionary
            logger.info(f"Successfully fetched data from the database using query from {query_file}")
            return data
        else:
            logger.error("No data found in the database")
            return None
    except Exception as e:
        logger.error(f"Error querying the database: {e}")
        return None

def fetch_recipients(recipients_file):
    # Load recipients from a plain text file
    try:
        with open(recipients_file, 'r') as file:
            recipients = [line.strip() for line in file if line.strip()]
        logger.info(f"Successfully loaded recipients from {recipients_file}")
        return recipients
    except Exception as e:
        logger.error(f"Error reading recipients file {recipients_file}: {e}")
        return []

def send_email(template_name, data_source, data_source_type, recipients_file):
    # Fetch recipients from the specified file
    recipients = fetch_recipients(recipients_file)
    if not recipients:
        logger.error("No valid recipients found.")
        return

    # Choose the data source type (JSON or DB)
    if data_source_type == 'json':
        data = fetch_data_from_json(data_source)
    elif data_source_type == 'db':
        data = fetch_data_from_db(data_source)
    else:
        logger.error("Invalid data source type. Use 'json' or 'db'.")
        return

    if not data:
        logger.error(f"No data found for the template {template_name}.")
        return

    # Create the email message
    msg = Message(f'{data.get("title", "Default Title")}',
                  recipients=recipients)

    # Render the HTML template using the fetched data
    msg.html = render_template(template_name, **data)

    try:
        mail.send(msg)
        logger.info(f'Email using {template_name} successfully sent to {len(recipients)} recipients!')
    except Exception as e:
        logger.error(f'Error sending email with template {template_name}: {e}')

if __name__ == '__main__':
    # Expect four arguments: template name, data source, data source type, and recipients file
    if len(sys.argv) > 4:
        template_name = sys.argv[1]
        data_source = sys.argv[2]
        data_source_type = sys.argv[3]  # 'json' or 'db'
        recipients_file = sys.argv[4]
        with app.app_context():
            logger.info(f"Starting email sending process with template {template_name}, data source {data_source}, data source type {data_source_type}, and recipients file {recipients_file}.")
            send_email(template_name, data_source, data_source_type, recipients_file)
            logger.info("Email sending process completed.")
    else:
        logger.error("Usage: python app.py <template_name> <data_source> <data_source_type> <recipients_file>")
        logger.info("Example: python app.py reminder_email.html data/reminder_data.json json recipients/recipients.txt")
        logger.info("Or: python app.py reminder_email.html data/reminder_query.sql db recipients/recipients.txt")


In [ ]:
data/json/reminder_data.json

In [ ]:
{
  "title": "Quarterly Attestation Reminder",
  "message": "This is a friendly reminder to complete your quarterly attestation form to ensure compliance with company policies.",
  "due_date": "September 30, 2024",
  "form_link": "https://example.com/attestation-form",
  "additional_info": "Please make sure to complete the form by the due date to avoid any compliance issues. Your cooperation is appreciated.",
  "footer": "If you have any questions or need assistance, please contact the compliance team at compliance@yourcompany.com.",
  "current_year": 2024
}


In [ ]:
data/sql/reminder_data.json.sql

In [ ]:
SELECT
    title,                -- Title of the email
    message,              -- Main body message of the email
    due_date,             -- Due date for completing the form
    form_link,            -- URL link to the attestation form
    additional_info,      -- Any extra information or instructions
    footer,               -- Footer text, like contact information
    strftime('%Y', 'now') AS current_year  -- Current year dynamically generated
FROM
    email_content
WHERE
    id = 1;               -- Adjust 'id' based on your data


In [ ]:
recipients/recipients.txt

In [ ]:
recipient1@example.com
recipient2@example.com
recipient3@example.com

In [ ]:
scripts/send_emails.sh

In [ ]:
#!/bin/bash

# Activate virtual environment if you are using one
source venv/bin/activate  # On Linux/macOS
# venv\Scripts\activate    # On Windows

# Send different types of emails with JSON data source
python app.py welcome_email.html data/json/email_data.json json recipients/recipients.txt

# Send email using a database query
python app.py password_reset.html data/sql/fetch_email_data.sql db recipients/recipients.txt

# Deactivate the virtual environment
deactivate


In [ ]:
templates/welcome_email.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ title }}</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            color: #333;
            margin: 0;
            padding: 20px;
        }
        .email-container {
            max-width: 600px;
            margin: 0 auto;
            background-color: #ffffff;
            border-radius: 5px;
            padding: 20px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        h1 {
            color: #333;
        }
        p {
            line-height: 1.6;
        }
        .button {
            display: inline-block;
            padding: 10px 20px;
            margin-top: 20px;
            background-color: #007bff;
            color: white;
            text-decoration: none;
            border-radius: 5px;
        }
        .footer {
            font-size: 0.8em;
            color: #666;
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <div class="email-container">
        <h1>{{ title }}</h1>
        <p>{{ message }}</p>
        <p><strong>Due Date:</strong> {{ due_date }}</p>
        <p>Please click the button below to complete your attestation form:</p>
        <a href="{{ form_link }}" class="button">Complete Form</a>
        <p>If the button above does not work, copy and paste the following URL into your browser:</p>
        <p>{{ form_link }}</p>
        <p>{{ additional_info }}</p>
        <div class="footer">
            <p>{{ footer }}</p>
            <p>&copy; {{ current_year }} Your Company. All rights reserved.</p>
        </div>
    </div>
</body>
</html>


In [ ]:
.env

In [ ]:
OUTLOOK_MAIL_USERNAME=your_email@outlook.com
OUTLOOK_MAIL_PASSWORD=your_password

In [ ]:
send_emails.sh

In [ ]:
chmod +x send_emails.sh

In [ ]:
./send_emails.sh